In [1]:
from datasets import load_dataset, DatasetDict, Dataset
from transformers import AutoTokenizer
import torch

/home/angkul/my_data/coding/agi/.mlenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = load_dataset("cfilt/iitb-english-hindi")

In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 1659083
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 520
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 2507
    })
})

In [4]:
# dataset['train'][0]['translation']['en']
train_subset = dataset['train']['translation'][:10]

In [5]:
dataset['train']['translation'][0]

{'en': 'Give your application an accessibility workout',
 'hi': 'अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें'}

In [17]:
for ex in dataset['train']['translation']:
    print(ex['en'])
    break

Give your application an accessibility workout


In [6]:
train_subset

[{'en': 'Give your application an accessibility workout',
  'hi': 'अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें'},
 {'en': 'Accerciser Accessibility Explorer',
  'hi': 'एक्सेर्साइसर पहुंचनीयता अन्वेषक'},
 {'en': 'The default plugin layout for the bottom panel',
  'hi': 'निचले पटल के लिए डिफोल्ट प्लग-इन खाका'},
 {'en': 'The default plugin layout for the top panel',
  'hi': 'ऊपरी पटल के लिए डिफोल्ट प्लग-इन खाका'},
 {'en': 'A list of plugins that are disabled by default',
  'hi': 'उन प्लग-इनों की सूची जिन्हें डिफोल्ट रूप से निष्क्रिय किया गया है'},
 {'en': 'Highlight duration', 'hi': 'अवधि को हाइलाइट रकें'},
 {'en': 'The duration of the highlight box when selecting accessible nodes',
  'hi': 'पहुंचनीय आसंधि (नोड) को चुनते समय हाइलाइट बक्से की अवधि'},
 {'en': 'Highlight border color',
  'hi': 'सीमांत (बोर्डर) के रंग को हाइलाइट करें'},
 {'en': 'The color and opacity of the highlight border.',
  'hi': 'हाइलाइट किए गए सीमांत का रंग और अपारदर्शिता। '},
 {'en': 'Highlight fill color', 'hi': 

In [7]:
hi_data = []
en_data = []

for i in range(len(train_subset)):
    hi_data.append(train_subset[i]['hi'])
    en_data.append(train_subset[i]['en'])

In [8]:
print(hi_data)

['अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें', 'एक्सेर्साइसर पहुंचनीयता अन्वेषक', 'निचले पटल के लिए डिफोल्ट प्लग-इन खाका', 'ऊपरी पटल के लिए डिफोल्ट प्लग-इन खाका', 'उन प्लग-इनों की सूची जिन्हें डिफोल्ट रूप से निष्क्रिय किया गया है', 'अवधि को हाइलाइट रकें', 'पहुंचनीय आसंधि (नोड) को चुनते समय हाइलाइट बक्से की अवधि', 'सीमांत (बोर्डर) के रंग को हाइलाइट करें', 'हाइलाइट किए गए सीमांत का रंग और अपारदर्शिता। ', 'भराई के रंग को हाइलाइट करें']


In [18]:
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-hi")

# Tokenize English sentences
en_tokenized = tokenizer(en_data, return_tensors="pt", padding=True, truncation=True)

# Tokenize Hindi sentences
hi_tokenized = tokenizer(hi_data, return_tensors="pt", padding=True, truncation=True)

# Encoder input (English sentences)
encoder_input_ids = en_tokenized["input_ids"]
encoder_attention_mask = en_tokenized["attention_mask"]

# Decoder input (Hindi sentences, shifted right by one token)
decoder_input_ids = hi_tokenized["input_ids"][:, :-1]  # Remove the last token
decoder_attention_mask = hi_tokenized["attention_mask"][:, :-1]

# Decoder target (Hindi sentences, shifted left by one token)
decoder_target_ids = hi_tokenized["input_ids"][:, 1:]  # Remove the first token

/home/angkul/my_data/coding/agi/.mlenv/lib/python3.10/site-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [20]:
from torch.utils.data import DataLoader, TensorDataset

# Create a TensorDataset
data = TensorDataset(encoder_input_ids, encoder_attention_mask, decoder_input_ids, decoder_attention_mask, decoder_target_ids)

# Create a DataLoader
batch_size = 8
dataloader = DataLoader(data, batch_size=batch_size, shuffle=True)